In [1]:
import os
import json
import pandas as pd
import numpy as np
from tqdm import tqdm
import spotipy
import sqlite3
from sqlite3 import Error
import matplotlib.pyplot as plt
from spotipy.oauth2 import SpotifyClientCredentials

In [2]:
def connect_db(db_file):
    conn = None
    try:
        conn = sqlite3.connect(db_file)
    except Error as e:
        print(e)
    return conn

In [55]:
def create_tables(conn):
    try:
        cur = conn.cursor()
        # create playlist table
        cur.execute("""CREATE TABLE IF NOT EXISTS playlists (
                                    name text NOT NULL,
                                    collaborative text,
                                    pid integer NOT NULL,
                                    modified_at integer,
                                    num_tracks integer,
                                    num_albums integer,
                                    num_followers integer,
                                    num_edits integer,
                                    duration_ms integer,
                                    num_artists integer
                                );""")
                                
        # create tracks table
        cur.execute(""" CREATE TABLE IF NOT EXISTS tracks (
                                    artist_name text,
                                    track_uri text NOT NULL,
                                    artist_uri text,
                                    track_name text NOT NULL,
                                    album_uri text,
                                    album_name text
                                    track_id integer
                                    pid integer
                                    ); """)
        cur.execute("""CREATE TABLE IF NOT EXISTS tracks_in_playlist (
                                    pid integer NOT NULL,
                                    track_uri
        );""")
        # create features table
        cur.execute(""" CREATE TABLE IF NOT EXISTS features_by_track (
                                    track_uri text primary key,
                                    danceability real,
                                    energy real,
                                    key real,
                                    loudness real,
                                    mode real,
                                    speechiness real,
                                    acousticness real,
                                    instrumentalness real,
                                    liveness real,
                                    valence real,
                                    tempo real,
                                    duration_ms integer,
                                    time_signature integer
                                    ); """)
        
        cur.execute(""" CREATE TABLE IF NOT EXISTS avg_features_by_playlist (
                                    pid integer NOT NULL primary key,
                                    danceability real,
                                    energy real,
                                    key real,
                                    loudness real,
                                    mode real,
                                    speechiness real,
                                    acousticness real,
                                    instrumentalness real,
                                    liveness real,
                                    valence real,
                                    tempo real,
                                    duration_ms integer,
                                    time_signature integer
        );""")

    except Error as e: 
        print(e)


In [4]:
def process_json_data(json_data, num_playlists, conn):
  # Get all playlists in the file
  playlists_df = pd.json_normalize(json_data['playlists'])
  playlists_df.drop(['tracks', 'description'], axis=1, inplace=True)
  playlists_df['name'].str.lower()
  playlists_df.to_sql(name='playlists', con=conn, if_exists='append', index=False)
  # Get all the tracks in the file
  cur = conn.cursor()
  cur.execute("select max(track_id) from tracks")
  rows = cur.fetchall()
  max_track_id = rows[0][0]
  if max_track_id is None:
      max_track_id = 0
  tracks_df = pd.json_normalize(json_data['playlists'], record_path=['tracks'], meta=['pid', 'num_followers'])
  #tracks_in_playlist_df.to_sql(name="tracks_in_playlist", con=conn, if_exists='append', index=False)
  tracks_df = tracks_df[tracks_df['pid'].isin(playlists_df['pid'].values)]
  tracks_df['track_uri'] = tracks_df['track_uri'].apply(lambda uri: uri.split(':')[2])
  tracks_df['album_uri'] = tracks_df['album_uri'].apply(lambda uri: uri.split(':')[2])
  tracks_df['artist_uri'] = tracks_df['artist_uri'].apply(lambda uri: uri.split(':')[2])

  tracks_in_playlist_df = tracks_df[['pid', 'track_uri']]
  tracks_in_playlist_df.to_sql(name='tracks_in_playlist', con=conn, if_exists='append', index=False)

  all_tracks_df = pd.read_sql('select track_id, track_uri from tracks', conn)
  tracks_df = tracks_df.merge(all_tracks_df, how='left', on='track_uri').fillna(0)
  tracks_df['track_id1'] = tracks_df[tracks_df["track_id"] == 0][['track_uri']].groupby('track_uri').ngroup()+max_track_id+1
  tracks_df['track_id'] = tracks_df['track_id'] + tracks_df['track_id1'].fillna(0)
  tracks_df['track_id'] = tracks_df['track_id'].astype('int64')
  tracks_df = tracks_df[tracks_df['track_id1'].notna()]
  tracks_df.drop(['pos', 'duration_ms', 'pid', 'num_followers', 'track_id1'], axis=1, inplace=True)
  tracks_df = tracks_df.drop_duplicates(subset='track_uri', keep="first")
  tracks_df.to_sql(name='tracks', con=conn, if_exists='append', index=False)

  


In [5]:
def process_playlists(path, num_files, num_playlists, conn):
    count = 0
    playlists = []
    filenames = os.listdir(path)
    for fname in sorted(filenames):
        if fname.startswith("mpd.slice.") and fname.endswith(".json"):
            count += 1
            full_path = os.sep.join((path, fname))
            with open(full_path) as f:
                js = json.loads(f.read())
                process_json_data(js, num_playlists, conn)
            if count == num_files and num_files > 0:
                break

In [56]:
path = 'data/spotify_million_playlist_dataset/data/'
conn = connect_db('data.db')

create_tables(conn)

In [57]:
process_playlists(path, 1, 0, conn)

In [8]:
def connect_to_spotify():
    # Spotify credentials
    cid = "5cffc2676cd44b35bc6af81faeb8e69a"
    secret = "f9fdae88362349b992ab2714ea91a094"
    os.environ["SPOTIPY_CLIENT_ID"] = cid
    os.environ["SPOTIPY_CLIENT_SECRET"] = secret
    os.environ['SPOTIPY_REDIRECT_URI'] = "http://localhost:8080"
    client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)
    sp = spotipy.Spotify(client_credentials_manager = SpotifyClientCredentials())

    return sp

In [73]:
def get_spotify_features():
    sp = connect_to_spotify()
    cur = conn.cursor()
    cur.execute('select track_id, track_uri from tracks')
    rows = cur.fetchall()
    uris = [row[1] for row in rows]
    feats_list = []
    for i in range(0, len(uris), 100):
        feats_list += sp.audio_features(uris[i:(i+100)])
    # Remove None items, for some tracks there are no features
    feats_list = [item for item in feats_list if item]
    features_df = pd.DataFrame(feats_list)
    features = ['danceability','energy','key','loudness','mode','speechiness','acousticness','instrumentalness','liveness','valence','tempo','duration_ms','time_signature']
    features_df = features_df[features]
    features_df.insert(loc=0, column='track_uri', value=uris)
    features_df.to_sql(name='features_by_track', con=conn, if_exists='append', index=False)

    # get the average features for each playlist and put into another table, along with the playlist name
    cur.execute('select tracks_in_playlist.pid, features_by_track.* from features_by_track join tracks_in_playlist on tracks_in_playlist.track_uri = features_by_track.track_uri')
    rows = cur.fetchall()

    # get the average features for each playlist
    cur.execute('''insert into avg_features_by_playlist select tracks_in_playlist.pid, AVG(danceability), AVG(energy), AVG('key'), AVG(loudness), AVG('mode'), AVG(speechiness),
                    AVG(acousticness), AVG(instrumentalness), AVG(liveness), AVG(valence), AVG(tempo), AVG(duration_ms), AVG(time_signature)
                    from features_by_track join tracks_in_playlist on tracks_in_playlist.track_uri = features_by_track.track_uri group by tracks_in_playlist.pid''')

    


In [74]:
get_spotify_features()


In [75]:
cur = conn.cursor()
cur.execute('select count(pid) from avg_features_by_playlist')
print(cur.fetchall())
cur.execute('select count(track_uri) from tracks_in_playlist group by pid')
print(len(cur.fetchall()))

[(1000,)]
1000
